# The goal of this program is to create a data frame that includes:
## Champion picks of every current Challenger player (including their climb games) **from season start**

In [1]:
import pandas as pd
import os
import cassiopeia as cass
import requests
import random
from ratelimit import limits
import json
import datetime
pwd = os.getcwd()

cass.set_riot_api_key("RGAPI-12d4c00a-e0b0-4c58-b857-45d1b4c0a725")

key = 'RGAPI-12d4c00a-e0b0-4c58-b857-45d1b4c0a725'

### Getting used to using Champion IDs & Returning Values into Arrrays

In [2]:
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region="NA")

summoner_name = []

for item in challenger_league:
    summoner_name.append(item.summoner.name)

print(summoner_name)


Making call: https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5
['FLY Yuuji', 'soupfan4', 'melke', '5tunt', 'Pinwinasio', 'admirable potato', 'MunchyTheMonster', 'Competition9', 'grey9', 'RHINOOOOOOOOOOOO', 'Pai Meí', '             Tçº', 'rallyracer21', 'spam tab', 'Blacc Massu', 'Modus Øperandi', 'cosha2', 'Nim Chimpsky', 'itdoesntmatter', 'Kevy C', 'Protos', 'Xiaojiang ovo', 'cranky enchanter', 'l7l', 'Jauny12', 'Szochs', 'Laceration', 'rehcra', 'Zyko', 'Aki Haruka', 'MUSTACHEmanFIGHT', 'An Nuo l', 'shxchi', 'tempmesyfnyiskkw', 'mohanno', 'Haru Urara', 'VX S9hua9', 'Cancoillotte', 'xusty', 'Mei You Qing Xu', 'rklckp251', 'QuinnAD Daddy', 'TL Bradley', 'TFBlade2', 'FLY Twoballs', 'bryan3838', 'hao de ', 'bizzy banks', 'FORWARDFORWARDFO', 'Nimdle', 'Peter Zezima', 'Tony Top', 'Ablazeolive', 'TL Hans samaa', 'McD Employee OTY', 'no invade pls', 'GURAA SHRIMP', 'Quantum', 'Dao Dwen Dwen', 'sigyl', 'bvoy3', 'Keel7', 'hoidro', 'Doublelift', 'ADC chai', '

In [3]:
summoner_puuid = []  

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    summoner_puuid.append(bucket.puuid)
    
summoner_puuid


Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/UyfPkJPa8k3Evf8rhMU9G2iAqEOMINsdTWoIpUUxiQ5GcmKWaAvY0vsj-w
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/1SjYuEP-mCZL1Ef92RcKBIwIFOzp5xIbuSzWO9I6oxbth1I
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/tqbCuoqDVaUqiGVWvtrdLJS6R8NwceReF4xzyOxXTHzN9Aea
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/wmnI4ZLn1yt4ZquA1NgWSHOBVW8dQeS-ntXTjwD177HzUnE
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/k8yeYzxQPlVwrHq7l2Al8_2DjcwsbK0-ACNCCFx0wEGIYv0
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/4J9if4NYkAkFaWsgOqO7R5Jpl69nkV_06YKzveQQK_iD
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/3_xROTs5g8GhD-bUn4IhrTFaUuHTz8MSduti9_K7r77B4_I
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/ffi9HBw7nSIvMi4XnzbkdNPwY7L_6_8k32Nm8AV_slk9Pno
Making call: https://na1.api.riotgames.com/lol/summoner

['uLisLErng2rjWsyMKOcXbAH1plaC4t6fPu6I3GjqDtQ0jHxoQ7uUYEDXr2Cswd0ISZb42tHvT0FNSA',
 '5_AdhP16jjsMni7wS-GDRq5b5GlXO-kjBAuI08g35EQ3atrXvObtthED0AbH-us_Hj6q3lB2v5L8XA',
 '8s2aj89cn6uXYzJVP4-f9e-pGil4a6TYsPIM4UegPGVpqf_1ARpk7LNNieHEmr8qntGV6o3296xjmg',
 'OlOuKUZ4ltpkLocL-7ga8xwzy20s9uYNiXSKlSdpNMdsYdX1wx2aiIfr5QjBG11RGX3qmq__qJZhzQ',
 'b1q7gp2umI65HqJc97z-QWjzYSzP8jI1fppHUYCF6L-IO2F3BRg4tVHr512xHltGbbFiSMRcsAfEzA',
 'qD0FnnCrtxOXwgITM2elVMQUpsJLuc2tY2zN8d2VX_p1IoX9RvKAi0_f9PWdB2cGCD_gYCeG96_eag',
 'ag1JpPrBqFdtrxdi8URUPlV9EL04H7gSaff5pZr2dPFir543sCoGuLi4qfiXZhUIT0pD6WMWZM90EQ',
 'yJcVjk1win2kCCPYPExupL5gIAC7Jqs8O6CSj55j7fOyoqmodE63ZlZA04n7y1jN_DRloMWjLAIOuw',
 'N9OGFft9Td5i4krX-Mwt9qiKYmmoZK4wr2sC_EOvQxXJnot4LqyyEgqNT_SpgtbSynUB_dR_MTA2_g',
 'B0dfpFMa16V83YoR3aE1XPykH6s5SzBleiSX60Tmt8pQFe6iXPsdfbSGIl6W9Px1r8HFDvN-JrZNLg',
 'AmPqEd3sAXX8znSrfOeBk2xOXIfJhtgOc9uWOq-VQCjZ8spjyRSRvWDGitB5Tg8a65rHNBN0GNhOPg',
 '-jCrH58C-3LMzurlrTtrMg-ehpwrWhEz31-oJJvUlDWx4ACSNL05AzWYW9MCnUAQg0_G5gbquXdZ0w',
 'j7

## Now that I have a list of summoner names and puuids, I can start pulling match histories

In [4]:
steve_history = []

steve = cass.Summoner(name='YoRHa Destiny', region='NA')
steve_history.append(cass.get_match_history(region=cass.data.Region.north_america,
                                             queue=cass.data.Queue.ranked_solo_fives, puuid=steve.puuid,
                                             continent=cass.data.Continent.americas))


         

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/YoRHa%20Destiny


In [5]:
print(steve)

steve_history[0][0].id

Summoner(id=cSFn_m6aqcr2x-oW1iC_TAuxUXUJKXNFylk_H1rGq6V9Euc, account_id=nGRaUbLW0jC1G5rBuctOfkQU-Z3vW3HaDCtgv9Y8owVe9g, name='YoRHa Destiny', puuid='9h-joR6pWHaTzM9wXXK8IV_nUU4GwM36C0lUvi7NSMw8HjPD1y3lGKrxiJqxmLCeH7X0HeGBF03dcQ')
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/9h-joR6pWHaTzM9wXXK8IV_nUU4GwM36C0lUvi7NSMw8HjPD1y3lGKrxiJqxmLCeH7X0HeGBF03dcQ/ids?beginIndex=0&endIndex=100&queue=420


'NA1_4222150635'

In [6]:


steve_champ = []

for item in steve_history[0]:
    steve_champ.append(item.participants[steve].champion.name)
    

steve_champ



Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222150635
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.4.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222104420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222080229
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4222033959
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221987338
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221993852
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221959657
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4221915701
Making call: https://americas.a

['Tristana',
 'Tristana',
 'Tristana',
 'Draven',
 'Draven',
 'Jinx',
 'Jinx',
 'Draven',
 'Jinx',
 'Draven',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx',
 'Jinx']

## Now that I kinda get how to pull match histories, I need to figure out how to store this list for all challenger players 

In [7]:
total_match_list = []

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    challenger_history = (cass.get_match_history(region=cass.data.Region.north_america,
                                             queue=cass.data.Queue.ranked_solo_fives, puuid=bucket.puuid,
                                             continent=cass.data.Continent.americas))
    total_match_list.append(challenger_history)


total_match_list

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/uLisLErng2rjWsyMKOcXbAH1plaC4t6fPu6I3GjqDtQ0jHxoQ7uUYEDXr2Cswd0ISZb42tHvT0FNSA/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/5_AdhP16jjsMni7wS-GDRq5b5GlXO-kjBAuI08g35EQ3atrXvObtthED0AbH-us_Hj6q3lB2v5L8XA/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/8s2aj89cn6uXYzJVP4-f9e-pGil4a6TYsPIM4UegPGVpqf_1ARpk7LNNieHEmr8qntGV6o3296xjmg/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/OlOuKUZ4ltpkLocL-7ga8xwzy20s9uYNiXSKlSdpNMdsYdX1wx2aiIfr5QjBG11RGX3qmq__qJZhzQ/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/b1q7gp2umI65HqJc97z-QWjzYSzP8jI1fppHUYCF6L-IO2F3BRg4tVHr512xHltGbbFiSMRcsAfEzA/ids?beginIndex=0&endIndex=100&queue=420
Making call: ht

[[<cassiopeia.core.match.Match at 0x217bb8fce20>,

In [8]:

total_match_list[0]

In [9]:

champion_picks = []
data = []
matches = []

for position, match_list in enumerate(total_match_list):
    bucket = cass.Summoner(name=summoner_name[position], region='NA')
    for match in match_list:
        champion = match.participants[bucket].champion.name
        champion_picks.append(champion)
        matches.append(match)
        tup = (bucket.name, champion, match)
        data.append(tup)

champion_picks
data        
matches

        


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4213346711
Making call: https://ddragon.leagueoflegends.com/cdn/12.3.1/data/en_US/championFull.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4212463394
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4210744837
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4210732062
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4210678801
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4210587793
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4210279000
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209689191
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209675824
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209681108
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA

 ...]

In [10]:
data.sort

data

[('FLY Yuuji', 'Zed', <cassiopeia.core.match.Match at 0x217bb8fce20>),
 ('FLY Yuuji', 'Kindred', <cassiopeia.core.match.Match at 0x217bba6b940>),
 ('FLY Yuuji', 'Xin Zhao', <cassiopeia.core.match.Match at 0x217bba6b220>),
 ('FLY Yuuji', 'Qiyana', <cassiopeia.core.match.Match at 0x217bba6b070>),
 ('FLY Yuuji', 'Twisted Fate', <cassiopeia.core.match.Match at 0x217bba6b820>),
 ('FLY Yuuji', 'Lee Sin', <cassiopeia.core.match.Match at 0x217bba6ba90>),
 ('FLY Yuuji', 'Xin Zhao', <cassiopeia.core.match.Match at 0x217bba6bb20>),
 ('FLY Yuuji', 'Xin Zhao', <cassiopeia.core.match.Match at 0x217bba6bbe0>),
 ('FLY Yuuji', 'Gwen', <cassiopeia.core.match.Match at 0x217bba6bca0>),
 ('FLY Yuuji', 'Ahri', <cassiopeia.core.match.Match at 0x217bba6bd60>),
 ('FLY Yuuji', 'Lee Sin', <cassiopeia.core.match.Match at 0x217bba6be20>),
 ('FLY Yuuji', 'Gwen', <cassiopeia.core.match.Match at 0x217bba6bee0>),
 ('FLY Yuuji', 'Ekko', <cassiopeia.core.match.Match at 0x217bba6bfa0>),
 ('FLY Yuuji', 'Viego', <cassiopei

In [11]:
dataframe = pd.DataFrame(data, columns= ['Summoner', 'Champion', 'Match'])

dataframe

,Summoner,Champion,Match
0,FLY Yuuji,Zed,"Match(id=4213346711, region='AMERICAS')"
1,FLY Yuuji,Kindred,"Match(id=4212463394, region='AMERICAS')"
2,FLY Yuuji,Xin Zhao,"Match(id=4210744837, region='AMERICAS')"
3,FLY Yuuji,Qiyana,"Match(id=4210732062, region='AMERICAS')"
4,FLY Yuuji,Twisted Fate,"Match(id=4210678801, region='AMERICAS')"
...,...,...,...
5995,rewinz1,Viego,"Match(id=4216350863, region='AMERICAS')"
5996,rewinz1,Viego,"Match(id=4215767575, region='AMERICAS')"
5997,rewinz1,Kha'Zix,"Match(id=4215426591, region='AMERICAS')"
5998,rewinz1,Xin Zhao,"Match(id=4215423216, region='AMERICAS')"


## I now know how pull matchlists, sort them by summoner, and store them into a data frame

### Now I need to figure out how to see which champions have the most picks, and test the things like:

#### "Most one trick", "Least one trick", "Highest Pick Rate for Climb", "Lowest Pick Rate for Climb" etc...

In [12]:
dataframe

dataframe_modified = dataframe.copy()

matches_set = set(matches)

match_count = len(matches_set)


In [13]:
dataframe_modified.describe()

,Summoner,Champion,Match
count,6000,6000,6000
unique,300,152,2908
top,FLY Yuuji,Jinx,"Match(id=4220781808, region='AMERICAS')"
freq,20,206,10


# Most Picked Champions

In [14]:
from collections import Counter

In [15]:
champ_pick_num = Counter(elem[1] for elem in data)

sorted_10  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=True)[:10]

sorted_10

for item in sorted_10:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Jinx: picked in 7.083906464924347% of games

Graves: picked in 5.433287482806053% of games

Ahri: picked in 4.883081155433287% of games

Jhin: picked in 4.814305364511692% of games

Aphelios: picked in 4.779917469050894% of games

Zeri: picked in 4.573590096286107% of games

Gwen: picked in 4.160935350756533% of games

Ezreal: picked in 4.092159559834938% of games

Viego: picked in 3.9202200825309492% of games

Viktor: picked in 3.7138927097661623% of games



# Least Picked Champions 
##### *With at least one pick*

In [16]:
sorted_10_bottom  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)[:10]

sorted_10_bottom

for item in sorted_10_bottom:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Kled: picked in 0.0343878954607978% of games

Warwick: picked in 0.0343878954607978% of games

Kog'Maw: picked in 0.0343878954607978% of games

Annie: picked in 0.0343878954607978% of games

Rammus: picked in 0.0343878954607978% of games

Master Yi: picked in 0.0343878954607978% of games

Yorick: picked in 0.0687757909215956% of games

Nocturne: picked in 0.0687757909215956% of games

Zyra: picked in 0.10316368638239339% of games

Ornn: picked in 0.10316368638239339% of games



# Champions Not Picked at All

In [17]:
picked_champs = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)

full_champ_list = []

list = cass.get_champions(region='NA')

for item in list:
    full_champ_list.append(item.name)

champions_not_picked = set(full_champ_list).difference(set(picked_champs))


print("These Champions were picked in 0 games:") 
for item in champions_not_picked:
    print('\n' + item)

These Champions were picked in 0 games:

Taric

Anivia

Illaoi

Vi

Vel'Koz

Skarner

Singed


# Most one-trick player

In [18]:
# How do I get the mode with my current tools [total_match_list, ('Summoner', 'Champion', 'Match'), summoner_name]
# My current plan is to use a dataframe to get the mode for each Summoner, I can also use the dataframe to get the total matches played per summoner. 

# ^This idea was good in my head but is total dogshit. I think I need to create some function to iterate through the list of tuples, and group them under their summoner name
#  (MAYBE THIS WAS SOMETHING THAT I SHOULD HAVE DONE DURING THE DATA COLLECTION PROCESS BUT HERE WE ARE, WE'RE TRYING TO LEARN EVEN IF ITS NOT OPTIMAL)



# I could then create a list of tuples (Player, Champion, Mode) 
# Not sure what to do afterward

# Perhaps create a parallel list of modes, and use a max function to pull the string, and then use that string to pull the Summoner and Champion index from the tuple

In [49]:


def Pull_Champ_Pick(summoner):
    success = None
    if summoner_name == item[0]:
        succeess = item[1]
    else:
        failure = None
        return succeess
        

for summoner in summoner_name:
    bucket = []
    for summoner in data:
        bucket.append(Pull_Champ_Pick(summoner))
    print(bucket)    








UnboundLocalError: local variable 'succeess' referenced before assignment

# Least one-trick player
